In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import lib

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import missingno as msno

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from sklearn.metrics import r2_score, accuracy_score, roc_auc_score


In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

# 2. Read data

In [ ]:
data_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
data_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

In [ ]:
data_train.head()

In [ ]:
data_test.head()

**Feature description**
* id: đánh dấu chỉ mục
* breath_id: chỉ mục cho hơi thở
* R: thuộc tính phổi cho biết mức độ hạn chế của đường thở (cmH2O/L/S). Ví dụ: khi thổi bóng bay bằng ông hút, ta gọi R là đường kính. Khi tăng R thì khó thổi
* C: thuộc tính phổi cho biết mức độ tuân thủ của phổi (Ml.cmH2O) - đây được gọi là sự thay đổi thể tích mỗi lần thay đổi áp suất. Ví dụ: Khi đổi C bằng cách thay đổi độ dày của quá bóng. Với C càng lớn thì độ mỏng của quả bóng càng nhiều và dễ thổi hơn
* time_step: thời gian thực tế
* u_in: Đầu vào của van điện tử cảm ứng. ( 0 <= u_in <= 100)
* u_out: Đầu vào cho van điện tử thăm dò. Có giá trị 0 hoặc 1
* pressure: áp lực đường thở đo bằng (cmH2O)

In [ ]:
# Train shape
row_train = data_train.shape[0]
column_train = data_train.shape[1]
print("Row train: \t\t", row_train)
print("Column train:\t\t", column_train)

In [ ]:
# Train shape
row_test = data_test.shape[0]
column_test = data_test.shape[1]
print("Row test: \t\t", row_test)
print("Column test:\t\t", column_test)

**Info data train**

In [ ]:
data_train.info()

**Info data test**

In [ ]:
data_test.info()

**Fill missing value**

In [ ]:
msno.bar(data_train, figsize = (20,3), fontsize = 12)
plt.grid();

In [ ]:
msno.bar(data_test, figsize = (20,3), fontsize = 12)
plt.grid();

> **=> No missing value**

**Remove Duplicate in data train**

In [ ]:
duplicate_train = data_train.duplicated()
print("Total duplicate value in Train dataset = %d" % duplicate_train.sum())

**Remove Duplicate in data test**

In [ ]:
duplicate_test = data_test.duplicated()
print("Total duplicate value in Test dataset = %d" % duplicate_test.sum())

In [ ]:
if duplicate_train.sum() == 0:
    print("No one data TRAIN is has been duplicated")
else:
    data_train[duplicate_train]
    

In [ ]:
if duplicate_test.sum() == 0:
    print("No one data TEST is has been duplicated")
else:
    data_train[duplicate_test]

# 3. Analysis data

In [ ]:
DF_train = data_train.copy()
DF_train.drop(['id', 'breath_id'], axis = 1, inplace = True)

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(DF_train.corr(), annot = True, fmt = '3.2f' , annot_kws={'size' : 12}, cmap="Set1")
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.show()

* Dựa vào đồ thị ta thấy time_step có mối liên quan trực tiếp với u_out
* Và pressure cũng có mổi tương quan với 2 đại lượng trên

In [ ]:
# Count unique value time_step
data_train.time_step.value_counts()

In [ ]:
# Count unique number of breath
unique_breaths = len(data_train.breath_id.unique())
print("No of Breaths:", unique_breaths)

# Group by ID
breath_length = data_train.groupby('breath_id').C.count().unique()
if len(breath_length) == 1:
    breath_length = breath_length[0]
print("Breath Length:", breath_length)

**Variance**

In [ ]:
# Tính độ chênh lệch của từng R C với giá trị trung bình R C
r_c_variance = data_train.groupby('breath_id')[['R','C']].var()
r_c_variance

=> Với mỗi breath R và C đều là hằng số duy nhất

In [ ]:
R_value = data_train[['breath_id', 'R']].groupby('breath_id').mean()['R']
print('Sự thay đổi duy nhất về áp suất(pressure) trên mỗi lần thay đổi lưu lượng khí (R):')
print(R_value.value_counts())
R = np.sort(R_value.unique()).astype(int)
print("R có", len(R) ,"giá trị là:", R)

In [ ]:
C_value = data_train[['breath_id', 'C']].groupby('breath_id').mean()['C']
print('Thay đổi thể tích(C) mỗi lần thay đổi áp suất(pressure):')
print(C_value.value_counts())
C = np.sort(C_value.unique()).astype(int)
print("C có", len(C) ,"giá trị là:", C)

**Kết hợp R và C**

In [ ]:
# Đếm số lượng R_C theo từng cặp
rc_values = np.array([
    [r, c, len(data_train[(data_train['R'] == r) & (data_train['C'] == c)])//breath_length] 
    for r in R 
    for c in C
])
x = range(len(rc_values))
plt.figure(figsize = (20,5))
plt.bar(x, rc_values[:,2])
plt.xticks(x, [str(r) + '_' + str(c) for r, c in rc_values[:,:2] ])
plt.xlabel('R_C')
plt.ylabel('Number counts')
plt.show()


**Biểu thị pressure, u_in, u_out**

In [ ]:
plt.figure(figsize = (20,12))
plt.subplot(211)
plt.plot(data_train.pressure[:2000], linewidth = 2, color = 'r', label="pressure")
plt.plot(data_train.u_in[:2000], linewidth = 2, color = 'g', label="u_in")
plt.plot(data_train.u_out[:2000], linewidth = 3, color = 'black', label="u_out")
plt.legend()

Biểu thị u_out

In [ ]:
plt.figure(figsize = (20,12))
plt.subplot(212)
plt.plot(data_train.u_out[:2000], linewidth = 3, label="u_out")
plt.legend()


* Ta có thể thấy rằng khi pressure(đường màu đỏ) tăng khi u_out = 0(Đường màu đen). và áp suất giảm khi u_out = 1
* u_in(đường màu xanh): ta có thể thấy khá tương quan với pressure - ta có thể thấy rõ ở biểu đồ bên trên

**Biểu thị pressure, time_step, u_out**

In [ ]:
plt.figure(figsize = (20,6))
plt.plot(data_train.time_step[:2000], linewidth = 2, color = 'r', label="time step")
plt.plot(data_train.pressure[:2000], linewidth = 2, color = 'g', label="pressure")
plt.plot(data_train.u_out[:2000], linewidth = 2, color = 'k', label="u_out")
plt.legend()
plt.legend()

plt.show()

**Select Input and Target Feature**
* id: Thuộc tính này k ảnh hưởng đến dự đoán pressure
* breath_id: Xác đinh sự kiện k ảnh hưởng đến dự đoán pressure
* R: Với mỗi chu kì thở, R không đổi, k có có sự thay đổi nào dc ghi nhận, vì thế thuộc tính cũng k cần thiểt cho dự đoán pressure
* C: Cũng tương tự như quan sát R, nến k cần thiết cho dự đoán pressure
* time_step: thuộc tính cũng không ý nghĩa cho dự đoán pressure
* **c_in: thuộc tính duy nhất tương quan đến pressure**
* **c_out: như công tắc trong quá trình tạo ra pressure( 0 || 1)**


In [ ]:
data_train_copy = data_train.copy()
data_train_copy.drop(columns=['id', 'breath_id', 'R', 'C', 'time_step'], axis = 1, inplace = True)

In [ ]:
data_train_copy.head()

In [ ]:
X = data_train_copy.drop(columns='pressure')
X = X[:80000]
y = data_train_copy['pressure']
y = y[:80000]

**Split data**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print('Dimension of X_train:', X_train.shape)
print('Dimension of X_test:', X_test.shape)

print('Dimension of y_train:', y_train.shape)
print('Dimension of y_test:', y_test.shape)

# 4. Build model 
**Linear Regression**

In [ ]:
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)

In [ ]:
LR_train_predict = LR_model.predict(X_train)
LR_test_predict = LR_model.predict(X_test)

In [ ]:
LR_R2_train = r2_score(y_train,LR_train_predict)
LR_R2_test = r2_score(y_test,LR_test_predict)

LR_MAPE_train = mape(y_train,LR_train_predict)
LR_MAPE_test = mape(y_test,LR_test_predict)

In [ ]:
resultsLR = pd.DataFrame({'R Squared Score': LR_R2_test,
                          'MAPE Score':LR_MAPE_test }
                           ,index=['Linear Regression'])

resultsLR

**Random forest**

In [ ]:
RF_model=RandomForestRegressor(n_estimators = 501, random_state=0)
RF_model.fit(X_train, y_train)

In [ ]:
RF_train_predict = RF_model.predict(X_train)
RF_test_predict = RF_model.predict(X_test)

In [ ]:
RF_R2_train = r2_score(y_train,RF_train_predict)
RF_R2_test = r2_score(y_test,RF_test_predict)

RF_MAPE_train = mape(y_train,RF_train_predict)
RF_MAPE_test = mape(y_test,RF_test_predict)


In [ ]:
resultsRF = pd.DataFrame({'R Squared Score': RF_R2_test,
                          'MAPE Score':RF_MAPE_test }
                           ,index=['Random Forest'])

results2 = pd.concat([resultsRF,resultsLR])
results2

In [ ]:
Submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
data_test.drop(columns=['id', 'breath_id', 'R', 'C', 'time_step'], axis = 1, inplace = True)

In [ ]:
Final_Prediction = LR_model.predict(data_test)


In [ ]:
Submission.pressure = Final_Prediction

In [ ]:
Submission.head(10)

In [ ]:
Submission2 = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
Final_Prediction2 = RF_model.predict(data_test)

In [ ]:
Submission2.pressure = Final_Prediction2

In [ ]:
Submission2.head(10)